# Mixed Effects Random Forest - Immune System Data

## Importing Packages

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from merf.merf import MERF
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import ParameterGrid
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
def load_obj(name ):                    
    with open('' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In the study the authors employed a LOPOCV methodology to address the fact that samples collected for each subject are dependent on their respective subject.

Another approach would be acknowledging the clustered nature of this data, and employ a suitable model where observations are dependent on the patient. Below I will implement a Mixed Effects Random Forest algorithm, clustered by patient.

Due to computational limits (this MERF package is quite intensive) I will only be utilizing the immune system dataset, and there will be extreme limitations put in place on cross validation as this package is not compatible with the more efficient GridSearchCV().

## Loading Data

In [98]:
df = datasets['immunesystem']
df = df[['_BL' not in x for x in df.index]] #fixing indices
df['response'] = datasets['response']
df.rename(columns={'Unnamed: 0':'PatientID'},inplace=True)
df['patient'] = list(range(1,18))*3
df['Z'] = np.ones(df.shape[0])
X = df.drop('response', axis=1)
y = df['response']

### Selecting Hyperparameters - Nested Cross Validation

Below is the top layer test/train split. The testing set will be left aside for model evaluation.

In [110]:
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(X, y, test_size=0.33, random_state = 42)

Below I split the training set to for hyperparameter selection.

In [111]:
import logging
logging.getLogger("merf").setLevel(logging.WARNING)
grid = {
            "n_estimators" : [100,200,300,400],
            "max_iterations" : [100,150],
        }
mselist = []
grids = []
X_train, X_test, y_train, y_test = train_test_split(X_train_meta, y_train_meta, test_size=0.33, random_state = 42)
for g in ParameterGrid(grid):
        train_clusters = X_train['patient']
        z_train = X_train[['Z']]
        X_train = X_train.drop(['patient','Z'], axis=1)
        z_test = X_test[['Z']]
        test_clusters = X_test['patient']
        X_test = X_test.drop(['patient','Z'], axis=1)
        mrf = MERF(**g)
        mrf.fit(X_train, z_train, train_clusters, y_train)
        predictions = mrf.predict(X_test, z_test, test_clusters)
        mse = mean_squared_error(y_test, predictions)
        mselist.append(mse)
        grids.append(g)

NameError: name 'best_score' is not defined

The error above was due to not saving the best score/grid correctly. Instead of re-running this code, I can simply observe the best values below:

In [114]:
grids

[{'max_iterations': 100, 'n_estimators': 100},
 {'max_iterations': 100, 'n_estimators': 200},
 {'max_iterations': 100, 'n_estimators': 300},
 {'max_iterations': 100, 'n_estimators': 400},
 {'max_iterations': 150, 'n_estimators': 100},
 {'max_iterations': 150, 'n_estimators': 200},
 {'max_iterations': 150, 'n_estimators': 300},
 {'max_iterations': 150, 'n_estimators': 400}]

In [115]:
mselist

[17.962749552396417,
 18.05286336323495,
 18.93322877699514,
 19.02728452950163,
 20.69524972008702,
 17.741120573519115,
 18.820842411308202,
 20.475879801385684]

Looking above, the MSE's look comparable. Due to computational limitations I will select the first grid corresponding to a MSE of ~17.96 as it will reduce runtime.

In [116]:
selected_parameters = {'max_iterations': 100, 'n_estimators': 100}

## Fitting the Model

Below I fit the model on the entirety of the training set, and evaluate its performance on the test set that was left aside.

In [117]:
train_clusters = X_train_meta['patient']
z_train = X_train_meta[['Z']]
X_train_meta = X_train_meta.drop(['patient','Z'], axis=1)
z_test = X_test_meta[['Z']]
test_clusters = X_test_meta['patient']
X_test_meta = X_test_meta.drop(['patient','Z'], axis=1)
mrf = MERF(**selected_parameters)
mrf.fit(X_train_meta, z_train, train_clusters, y_train_meta)

In [119]:
predictions = mrf.predict(X_test_meta, z_test, test_clusters)
mse = mean_squared_error(y_test_meta, predictions)
print(mse)

26.234772335300683


The model achieved a relatively low MSE! However this result must be taken with a large grain of salt, as it is entirely dependent on each train/test split. A different split (modifying random_state), could give a much different result. 